<div class="alert alert-block alert-info">
<h3> <b> Part 4. Data Munging <b></h3>
<div>

❓ **Your challenge**: 

- Use your best judgement to determing the join conditions for the data sets you cleand already: `flights_cleaned`, `tickets_cleaned`, `airport_codes_cleaned`
- Read the case carefully, you may need to figure out how to calculate certain KPIs in a reasonable way

💡 Suggested methodology:
- Use the notebook below to write and test your code step-by-step first
- Create any functions and put them in a .py file to join data if necessary 

In [1]:
# Add any packages you need here
import pyforest
import copy
import string
import missingno as msno
import requests
import zipfile
import os

from Airport.get_data import GetData
from Airport import check_data, fix_data

import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", lambda x: "%.2f" %x) # suppress scientific notation

# This can help to autoreload the packages you create
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<div class="alert alert-block alert-success">
<h4> <b> 1. Import Data <b></h4>
<div>

👉 Import three datasets- `flights_cleaned`, `tickets_cleaned`, `airport_codes_cleaned` we cleaned in the previous section for later usage, you can discard attributes that you identified unnecessary for further analysis

In [3]:
# Import the datasets - write your code here
flights = pd.read_csv(os.path.join("Raw Data", "flights_cleaned.csv"))
tickets = pd.read_csv(os.path.join("Raw Data", "tickets_cleaned.csv"))
airport_codes = pd.read_csv(os.path.join("Raw Data", "airport_codes_cleaned.csv"))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
airport_codes = airport_codes[["IATA_CODE", "TYPE"]].drop_duplicates()

<div class="alert alert-block alert-success">
<h4> <b> 2.Data Munging <b></h4>
<div>

#### a) Flights

👉 Follow the instructions below to enrich your `flights` data by generating new features:

> 1. Add a new feature `ROUND_TRIP_ROUTE` - An example of a round
trip route is the combination of JFK to ORD and ORD to JFK. The opposite order of the
route, ORD to JFK and JFK to ORD, would be considered the same round trip

In [6]:
flights['ROUND_TRIP_ROUTE'] = flights.apply(lambda x: '-'.join(set([x['ORIGIN'], x['DESTINATION']])), axis = 1)

In [189]:
flights["ROUND_TRIP_ROUTE"] = \
            np.where(flights['ORIGIN'] < flights['DESTINATION'], \
                     flights['ORIGIN'] + '-' + flights['DESTINATION'], \
                     flights['DESTINATION'] + '-' + flights['ORIGIN'])

<IPython.core.display.Javascript object>

> 2. Add a new feature `COST_BASE`, which comprises of cost of:
-  Fuel, Oil, Maintenance, Crew 8 USD per mile total
- Depreciation, Insurance, Other 1.18 USD per mile total

In [7]:
# Fuel, Oil, Maintenance, Crew + Depreciation, Insurance, Other Costs
flights["COST_BASE"] = flights["DISTANCE"] * (8 + 1.18)

> 3. Add a new feature `COST_AIRPORT_OP`, which stands for Airport operational costs for the right to use the airports and related services are fixed at 5,000 USD for medium airports and $10,000 for large airports. There is one charge for each airport where a flight lands. Thus, a round trip flight has a total of two airport charges

In [8]:
flights = flights.\
            merge(airport_codes.add_prefix("ORIGIN_"), \
                  left_on = "ORIGIN", right_on = "ORIGIN_IATA_CODE").\
            merge(airport_codes.add_prefix("DEST_"), \
                  left_on = "DESTINATION", right_on = "DEST_IATA_CODE")

In [9]:
# Airport operational costs
flights["COST_AIRPORT_OP"] = \
            np.where(flights["ORIGIN_TYPE"] == "large_airport", 10000, 5000) + \
            np.where(flights["DEST_TYPE"] == "large_airport", 10000, 5000)

flights["COST_AIRPORT_OP"] = flights["COST_AIRPORT_OP"]/2

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

> 4. Add a new feature `COST_DELAY`, Delays that are 15 minutes or less are free, however each additional minute of
delay costs the airline $75 in added operational costs. This is charged separately
for both arrival and departure delays

<details>
    <summary>💡Hint </summary>

You may need to use information from `airport_codes` dataset
</details>

In [10]:
# Delay costs
flights["COST_DELAY_DEP"] = \
            np.where(flights["DEP_DELAY"] <= 15, 0, 75 * (flights["DEP_DELAY"] - 15))

flights["COST_DELAY_ARR"] = \
            np.where(flights["ARR_DELAY"] <= 15, 0, 75 * (flights["ARR_DELAY"]- 15))      

flights["COST_DELAY"] = flights["COST_DELAY_DEP"] + flights["COST_DELAY_ARR"]
flights.drop(["COST_DELAY_DEP", "COST_DELAY_ARR"], axis = 1, inplace = True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### b) Tickets

👉 Follow the instructions to process `tickets` dataset:

> 1. Add a new feature `ROUND_TRIP_ROUTE` - An example of a round
trip route is the combination of JFK to ORD and ORD to JFK. The opposite order of the
route, ORD to JFK and JFK to ORD, would be considered the same round trip

In [11]:
tickets["ROUND_TRIP_ROUTE"] = \
        np.where(tickets['ORIGIN'] < tickets['DESTINATION'], \
                 tickets['ORIGIN'] + '-' + tickets['DESTINATION'], \
                 tickets['DESTINATION'] + '-' + tickets['ORIGIN'])

<IPython.core.display.Javascript object>

> 2. Calculate median ticket price (`ITIN_FARE`) for each `ROUND_TRIP_ROUTE`

In [12]:
tickets_price = tickets.groupby("ROUND_TRIP_ROUTE").agg({"ITIN_FARE": "median"}).reset_index()

#### c) Calculate Route Cost, Revenue and Profit

> 1. Add information of ticket price (`ITIN_FARE`) for each `ROUND_TRIP_ROUTE` to the `flights` dataset

In [13]:
_ = flights.merge(tickets_price)

> 2. Calculate the following Revenues and Costs based on business assumptions:

- `REVENUE_TICKETS`: Each plane can accommodate up to 200 passengers and each flight has anassociated occupancy rate provided in the Flights data set. Do not use the Tickets data set to determine occupancy
- `REVENUE_BUGGAGE`: Baggage fee is 35 USD for each checked bag per flight. We expect 50% of passengers to check an average of 1 bag per flight. The fee is charged separately for each leg of a round trip flight, thus 50% of passengers will be charged a total of 70 USD in baggage fees for a round trip flight
- `COST_TOTAL`: Sum of all the costs for the round trip route
- `REVENUE_TOTAL`: Sum of all the revenues for the round trip route
- `PROFIT`: `REVENUE_TOTAL` - `COST_TOTAL`

In [15]:
_ ["REVENUE_TICKETS"] = 200 * _ ["OCCUPANCY_RATE"] * _ ["ITIN_FARE"] * 0.5
_ ["REVENUE_BUGGAGE"] = 200 * _ ["OCCUPANCY_RATE"] * 0.5 * 35
_["COST_TOTAL"] = _["COST_BASE"] + _["COST_AIRPORT_OP"] + _["COST_DELAY"]
_["REVENUE_TOTAL"] = _["REVENUE_TICKETS"] + _["REVENUE_BUGGAGE"]
_["PROFIT"] = _["REVENUE_TOTAL"] - _["COST_TOTAL"]

> 3. Generate a summary P&L (Profit & Loss) table containing the follwing attributes for each round trip route:

- `DEP_DELAY`: median of the DEO_DELAY
- `ARR_DELAY`: median of the ARR_DELAY
- `AIR_TIME`: median of the AIR_TIME
- `DISTANCE`: median of the DISTANCE
- `OCCUPANCY_RATE`: median of the OCCUPANCY_RATE
- `TRIP_FREQ`: number of trips
- `COST_BASE`: sum of COST_BASE
- `COST_AIRPORT_OP`:  sum of COST_AIRPORT_OP
- `COST_DELAY`: sum of COST_DELAY
- `REVENUE_TICKETS`: sum of REVENUE_TICKETS
- `REVENUE_BUGGAGE`: sum of REVENUE_BUGGAGE
- `COST_TOTAL`: sum of COST_TOTAL
- `REVENUE_TOTAL`: sum of REVENUE_TOTAL
- `PROFIT`: sum of PROFIT

In [17]:
result = _.groupby("ROUND_TRIP_ROUTE").agg({"DEP_DELAY": "median",
                                            "ARR_DELAY": "median",
                                            "AIR_TIME": "median",
                                            "DISTANCE": "median",
                                            "OCCUPANCY_RATE": "median",
                                            "FL_DATE": "count",
                                            "COST_BASE": "sum",
                                            "COST_AIRPORT_OP": "sum",
                                            "COST_DELAY": "sum",
                                            "REVENUE_TICKETS": "sum",
                                            "REVENUE_BUGGAGE": "sum",
                                            "COST_TOTAL": "sum",
                                            "REVENUE_TOTAL": "sum",
                                            "PROFIT": "sum"}).\
                                    rename(columns = {"FL_DATE": "TRIP_FREQ"}).reset_index()

<div class="alert alert-block alert-success">
<h4> <b> 3. Save P&L Table <b></h4>
<div>

👉 Save the summarized P&L table in the in the `Result` folder for further business analysis

In [19]:
# Write your code here
if os.path.exists("Result"):
    result_file_path = os.path.join(os.getcwd(), "Result")
    print("Result file path is:", result_file_path)
else: 
    os.mkdir("Result")
    print("Result folder created")

result.to_csv(os.path.join(result_file_path, "profit_and_loss.csv"), index = False)

Result file path is: F:\Anaconda Files\Datark\商业分析实战项目\Code\Result
